In [ ]:
!pip install langchain langgraph langchain-google-genai langchain-community


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.7/87.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.9/394.9 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfu

In [ ]:
import json
from typing import Annotated, Literal, TypedDict

from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ChatMessage
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode
import json
from typing import List, Dict, Any, Optional, Callable, Tuple, Sequence
from langchain_core.messages import HumanMessage, SystemMessage
import uuid
import time
from collections import defaultdict
import pprint

from langchain_community.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper

In [ ]:
#with open("secrets.json", "r") as f:
#    api_key = json.loads(f.read())["google-key"]
from google.colab import userdata
api_key = userdata.get('google')


In [ ]:
def concatenate(*iters):
    for it in iters:
        yield from it

In [ ]:
from typing import List, Dict, Any, Optional, Callable, Tuple, Sequence
from langchain_core.messages import HumanMessage, SystemMessage
import uuid
import time



class InteractionGraph:
    def __init__(self):
        self.nodes: Dict[str, Dict[str, List[Tuple[float, str]]]] = defaultdict(lambda: defaultdict(list))

    def add_interaction(self, sender: str, recipient: str, message: str):
        """Add an interaction between two agents."""
        timestamp = time.time()
        # Check if this exact message already exists

        self.nodes[sender][recipient].append((timestamp, message))

    def add_llm_call(self, agent: str, response: str):
        """Add an LLM call for a specific agent."""
        timestamp = time.time()
        self.nodes[agent]["LLM"].append((timestamp, response))

    def get_all_interactions(self, agent: str) -> List[Tuple[float, str, str, str]]:
        """Get all interactions for a specific agent."""
        interactions = []
        for recipient, messages in self.nodes[agent].items():
            for timestamp, message in messages:
                interactions.append((timestamp, agent, recipient, message))
        for sender, recipients in self.nodes.items():
            if agent in recipients:
                for timestamp, message in recipients[agent]:
                    interactions.append((timestamp, sender, agent, message))
        return sorted(interactions, key=lambda x: x[0])  # Sort by timestamp

    def get_memory_messages(self, agent: str) -> List[Any]:
        """Get a list of messages representing all interactions for a specific agent."""
        interactions = self.get_all_interactions(agent)
        messages = []
        for _, sender, recipient, content in interactions:
            if recipient == "LLM":
                messages.append(AIMessage(content=content))
            elif sender == agent:
                messages.append(HumanMessage(content=f"Recipient_id: {recipient} | Sender_id: {sender} -> {content}"))
            else:
                messages.append(HumanMessage(content=f"Recipient_id: {recipient} | Sender_id: {sender} -> {content}"))
        return messages

    def get_memory_string(self, agent: str) -> str:
        """Get a formatted string of all interactions for a specific agent."""
        messages = self.get_memory_messages(agent)
        return "\n".join([f"{msg.__class__.__name__}: {msg.content}" for msg in messages])

class Agent:
    def __init__(self, agent_id: str = None, workflow = None, description = None, tools = None, starting_task=None):
        if agent_id is None:
            agent_id = str(uuid.uuid4())
        self.agent_id = agent_id
        self.messages = {}
        self.environment = None
        self.workflow = workflow
        self.model = None
        self.description = description if description else "No description provided"
        self.tools = tools if tools else []
        self.starting_task = starting_task if starting_task else "No starting task provided"
        self.memory_graph = InteractionGraph()

        self.memory_graph.add_interaction("SYSTEM", self.agent_id, self.starting_task)


    def receive_message(self, message: str, sender_id: str):
        msg = {"sender": sender_id, "message": message, "time": time.time(), "recipient": self.agent_id}
        if sender_id in self.messages:
            self.messages[sender_id].append(msg)
        else:
            self.messages[sender_id] = [msg]
        self.memory_graph.add_interaction(sender_id, self.agent_id, message)
        return self.execute(sender_id)

    def send_message(self, recipient_id: str, message: str):
        environment = self.environment
        msg = {"sender": self.agent_id, "message": message, "time": time.time(), "recipient": recipient_id}
        if recipient_id in self.messages:
            self.messages[recipient_id].append(msg)
        else:
            self.messages[recipient_id] = [msg]
        environment.send_message(self.agent_id, recipient_id, message)
        self.memory_graph.add_interaction(self.agent_id, recipient_id, message)
    def execute_all(self):
        #print(self.build_message_all())
        final_state = self.workflow.invoke(
            {"messages": self.build_message_all()},
            config={"configurable": {"thread_id": 42}}
        )
        last_message = final_state["messages"][-1]
        self.memory_graph.add_llm_call(self.agent_id, last_message.content)
        return last_message

    def execute(self, agent_id: str):
        messages = self.build_message(agent_id)
        final_state = self.workflow.invoke(
            {"messages": messages},
            config={"configurable": {"thread_id": 42}}
        )
        last_message = final_state["messages"][-1]
        self.memory_graph.add_llm_call(self.agent_id, last_message.content)

        return last_message

    def run(self):
        self.execute_all()

    def build_message_all(self):
        memory_string = self.memory_graph.get_memory_string(self.agent_id)
        return [HumanMessage(content=f"{self.starting_task}\n\nCurrent memory:\n{memory_string}")]

    def build_message(self, agent_id):
        memory_string = self.memory_graph.get_memory_string(self.agent_id)
        return [HumanMessage(content=f"{self.starting_task}\n\nCurrent memory:\n{memory_string}")]


    def bind_environment(self, environment):
        self.environment = environment

    def bind_model(self, model):
        self.model = model

    def bind_workflow(self, workflow):
        self.workflow = workflow



def get_tool(assistant_id: str) -> Dict[str, Any]:
    return {
        "search_agents": search_factory(assistant_id=assistant_id),
    }

def send_factory(agent):
    @tool
    def send_message(agent_id: str, query: str) -> bool:
        """Sends a message to an agent, using agent_id inside the environment"""
        agent.memory_graph.add_interaction(agent.agent_id, agent_id, query)
        return agent.send_message(agent_id, query)

    return send_message

def search_factory(agent):
    @tool
    def search_agents(query: str) -> str:
        """Searches for different agents inside an environment, given a query. Returns a list of agents with their id and a description, in JSON format"""
        agents = agent.environment.search_agent(query, agent)
        json_result = json.dumps(agents)
        agent.memory_graph.add_interaction(agent.agent_id, "SYSTEM", f"Searched for agents: {query}\nResult: {json_result}")
        return json_result

    return search_agents

def call_factory(agent):
    def call_model(state: MessagesState):
        time.sleep(8)
        messages = agent.build_message(agent.agent_id)
        print('\n\n', agent.agent_id)
        pprint.pp([x.content for x in messages])
        print(len(" ".join([str(x.content) for x in messages]).split()))
        print("\n\n")
        response = agent.model.invoke(messages)


        response_content = response.content if (hasattr(response, 'content') and response.content) else str(response)

        # Store only the LLM response content in the memory graph
        agent.memory_graph.add_llm_call(agent.agent_id, response_content)

        return {"messages": response}

    return call_model
class Environment:
    def __init__(self, no_self_send = True):
        self.agents: Dict[str, Agent] = {}
        self.no_self_send = no_self_send

    def add_agent(self, agent: Agent):
        self.agents[agent.agent_id] = agent
        agent.bind_environment(self) #maybe add the factories to tools too


    def remove_agent(self, agent_id: str):
        if agent_id in self.agents:
            del self.agents[agent_id]

    def get_agent(self, agent_id: str) -> Agent|None:
        return self.agents.get(agent_id, None)

    def get_all_agents(self) -> List[Agent]:
        return list(self.agents.values())

    def send_message(self, sender_id: str, recipient_id: str, message: str, delay=None):
        if delay:
            time.sleep(delay)
        if sender_id in self.agents and recipient_id in self.agents:
            if sender_id == recipient_id and self.no_self_send:
                return
            else:
                self.agents[recipient_id].receive_message(message, sender_id)
        else:
            print(f"Error: Invalid sender or recipient ID s:{sender_id} r:{recipient_id}")

    def broadcast_message(self, sender_id: str, message: str):
        for agent_id, agent in self.agents.items():
            if agent_id != sender_id:
                agent.receive_message(message, sender_id)
    def search_agent(self, query:str, agent = None) -> str:
      agent_id = agent.agent_id if agent else None
      return [{"id":a.agent_id, "description":a.description} for a in self.get_all_agents() if a.agent_id!=agent_id]





env = Environment()



In [ ]:
class CycleStopper:
  def __init__(self, max_cycles: int = 5, agent_name="agent1"):
    self.max_cycles = max_cycles
    self.current = 0
    self.agent_name = agent_name
  def __call__(self, state: MessagesState):
    self.current += 1
    if self.current >= self.max_cycles:
      return END
    return self.agent_name

def restrict(state: MessagesState) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1]
    # If the LLM makes a tool call, then we route to the "tools" node
    if last_message.tool_calls:
        return "tools"
    # Otherwise, we stop (reply to the user)
    return END
def is_tool_call(state: MessagesState) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1]

    if last_message.tool_calls:
        return "tools"
    return END


def summarize(state: MessagesState):
    messages = state['messages']

    response = model.invoke(messages+[HumanMessage("Summarize the content")])

    return {"messages": [response]}


In [ ]:

agent_buy = Agent("agent_buy", description="An agent that purchases food for a customer", starting_task="You are an agent with access to tools, and who will be interacting with other agents. You will receive different messages from different agents to complete a task. You have access to an environment to interact with other agents. Any missing information might be found by searching for an agent that has it. After searching for agents you may connect to them using their id. It is possible that the agent does not exist, so after trying once, try to solve it without communication, and retry after. The task is: Order a hamburger")
tools = [search_factory(agent_buy), send_factory(agent_buy)]
tool_node = ToolNode(tools)
model_buy = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.2, google_api_key=api_key).bind_tools(tools)
agent_buy.bind_model(model_buy)
workflow_buy = StateGraph(MessagesState)
workflow_buy.add_node("agent", call_factory(agent_buy))
workflow_buy.add_node("tools", tool_node)
workflow_buy.set_entry_point("agent")
workflow_buy.add_edge("agent", "tools")
cycle_stopper = CycleStopper(agent_name="agent", max_cycles=9)
workflow_buy.add_conditional_edges(
    "tools",
    cycle_stopper,
)
checkpointer = MemorySaver()
app = workflow_buy.compile(checkpointer=checkpointer)
agent_buy.bind_workflow(app)
env.add_agent(agent_buy)

In [ ]:
@tool
def give_user_info(query: str):
  """Returns the info of a particular user"""
  return json.dumps({"credit card": "5535262626346",
                     "food preference": "Ham burger with cheese",
                     "name": "Jason Smith",
                     "address": "cr 34 #352-532",
                     })

agent_detail = Agent("agent_detail", description="An agent that provides information regarding user information (cc, food preference, name, address)", starting_task="Provide user information when asked. You might contact the agent using the sender_id to provide information, never to ask for information. always start by calling give_user_info, to get the user info and then contact the other agent")
tools_detail  = [ give_user_info,  send_factory(agent_detail)]
tool_node_detail  = ToolNode(tools_detail )
model_detail = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.2, google_api_key=api_key).bind_tools(tools_detail )
agent_detail.bind_model(model_detail)
workflow_detail = StateGraph(MessagesState)
workflow_detail.add_node("agent", call_factory(agent_detail))
workflow_detail.add_node("tools", tool_node_detail)
workflow_detail.set_entry_point("agent")
workflow_detail.add_edge("agent", "tools")
cycle_stopper = CycleStopper(agent_name="agent", max_cycles=7)
workflow_detail.add_conditional_edges(
    "tools",
    cycle_stopper,
)
checkpointer = MemorySaver()
app = workflow_detail.compile(checkpointer=checkpointer)
agent_detail.bind_workflow(app)
env.add_agent(agent_detail)

In [ ]:
agent_sell = Agent("agent_sell", description="An agent that sells various items", starting_task="You are an agent with access to tools, and who will be interacting with other agents. You will receive different messages from different agents to complete a task. You have access to an environment to interact with other agents. Any missing information might be found by searching for an agent that has it. After searching for agents you may connect to them using their id. It is possible that the agent does not exist, so after trying once, try to solve it without communication, and retry after. Your task is to successfully sell various food items to other agents")
tools = [search_factory(agent_sell ), send_factory(agent_sell)]
tool_node = ToolNode(tools)
model_sell = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.2, google_api_key=api_key).bind_tools(tools)
agent_sell .bind_model(model_sell)
workflow_sell = StateGraph(MessagesState)
workflow_sell.add_node("agent", call_factory(agent_sell))
workflow_sell.add_node("tools", tool_node)
workflow_sell.set_entry_point("agent")
workflow_sell.add_edge("agent", "tools")
cycle_stopper = CycleStopper(agent_name="agent", max_cycles=9)
workflow_sell.add_conditional_edges(
    "tools",
    cycle_stopper,
)
checkpointer = MemorySaver()
app = workflow_sell.compile(checkpointer=checkpointer)
agent_sell.bind_workflow(app)
env.add_agent(agent_sell)

In [ ]:
f = agent_buy.run()

Se han truncado las últimas 5000 líneas del flujo de salida.
 'HumanMessage: Recipient_id: SYSTEM | Sender_id: agent_sell -> Searched for '
 'agents: agent with price list\n'
 'Result: [{"id": "agent_buy", "description": "An agent that purchases food '
 'for a customer"}, {"id": "agent_detail", "description": "An agent that '
 'provides information regarding user information (cc, food preference, name, '
 'address)"}]\n'
 "AIMessage: content='' additional_kwargs={'function_call': {'name': "
 '\'send_message\', \'arguments\': \'{"agent_id": "agent_detail", "query": '
 '"What is the price of a hamburger?"}\'}} '
 "response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': "
 "[]}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': "
 "'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, "
 "{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', "
 "'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', "
 "'probabi

In [ ]:
agent_buy.messages

{'agent_sell': [{'sender': 'agent_buy',
   'message': 'I would like to order a hamburger',
   'time': 1724820463.571808,
   'recipient': 'agent_sell'},
  {'sender': 'agent_sell',
   'message': 'The price of a hamburger is $5. Would you like to purchase it?',
   'time': 1724820600.39749,
   'recipient': 'agent_buy'},
  {'sender': 'agent_buy',
   'message': 'Yes, please.',
   'time': 1724820608.9283156,
   'recipient': 'agent_sell'},
  {'sender': 'agent_sell',
   'message': 'Your hamburger is ready. Please come pick it up.',
   'time': 1724820619.09369,
   'recipient': 'agent_buy'},
  {'sender': 'agent_buy',
   'message': 'What is your address?',
   'time': 1724820653.2728071,
   'recipient': 'agent_sell'},
  {'sender': 'agent_sell',
   'message': 'The address of the customer is cr 34 #352-532. Please come pick up your order.',
   'time': 1724820714.202412,
   'recipient': 'agent_buy'},
  {'sender': 'agent_buy',
   'message': 'I will go pick it up now.',
   'time': 1724820722.850829,
   

In [ ]:
agent_sell.messages

{'agent_buy': [{'sender': 'agent_buy',
   'message': 'I would like to order a hamburger',
   'time': 1724820463.5718153,
   'recipient': 'agent_sell'},
  {'sender': 'agent_sell',
   'message': 'The price of a hamburger is $5. Would you like to purchase it?',
   'time': 1724820600.3974807,
   'recipient': 'agent_buy'},
  {'sender': 'agent_buy',
   'message': 'Yes, please.',
   'time': 1724820608.9283364,
   'recipient': 'agent_sell'},
  {'sender': 'agent_sell',
   'message': 'Your hamburger is ready. Please come pick it up.',
   'time': 1724820619.0936797,
   'recipient': 'agent_buy'},
  {'sender': 'agent_buy',
   'message': 'What is your address?',
   'time': 1724820653.2728162,
   'recipient': 'agent_sell'},
  {'sender': 'agent_sell',
   'message': 'The address of the customer is cr 34 #352-532. Please come pick up your order.',
   'time': 1724820714.2024033,
   'recipient': 'agent_buy'},
  {'sender': 'agent_buy',
   'message': 'I will go pick it up now.',
   'time': 1724820722.850837

In [ ]:
agent_detail.messages

{'agent_sell': [{'sender': 'agent_sell',
   'message': 'What is the price of a hamburger?',
   'time': 1724820480.5934875,
   'recipient': 'agent_detail'},
  {'sender': 'agent_sell',
   'message': 'What is the price of a hamburger?',
   'time': 1724820557.1506555,
   'recipient': 'agent_detail'},
  {'sender': 'agent_sell',
   'message': 'What is the price of a hamburger?',
   'time': 1724820582.9319825,
   'recipient': 'agent_detail'},
  {'sender': 'agent_detail',
   'message': 'The price of a hamburger is $5',
   'time': 1724820591.6892357,
   'recipient': 'agent_sell'},
  {'sender': 'agent_sell',
   'message': 'What is the address of the customer?',
   'time': 1724820670.4966812,
   'recipient': 'agent_detail'},
  {'sender': 'agent_sell',
   'message': 'What is the address of the customer?',
   'time': 1724820696.5049815,
   'recipient': 'agent_detail'},
  {'sender': 'agent_detail',
   'message': 'The address of the customer is cr 34 #352-532',
   'time': 1724820705.3112307,
   'reci

In [ ]:
agent_detail.memory_graph.nodes

defaultdict(<function __main__.InteractionGraph.__init__.<locals>.<lambda>()>,
            {'SYSTEM': defaultdict(list,
                         {'agent_detail': [(1724820104.4288764,
                            'Provide user information when asked. You might contact the agent using the sender_id to provide information, never to ask for information. always start by calling give_user_info, to get the user info and then contact the other agent')]}),
             'agent_buy': defaultdict(list,
                         {'agent_detail': [(1724820107.746329,
                            'What is my address?')]}),
             'agent_detail': defaultdict(list,
                         {'LLM': [(1724820108.4027557, ''),
                           (1724820108.8719974, ''),
                           (1724820109.3102436, ''),
                           (1724820109.8454888, ''),
                           (1724820110.2762496, ''),
                           (1724820110.7098181, ''),
              

In [ ]:
@tool
def give_user_info(query: str):
  """Returns the info of a particular user"""
  return json.dumps({"credit card": "5535262626346",
                     "food preference": "Ham burger with cheese",
                     "name": "Jason Smith",
                     "address": "cr 34 #352-532",
                     })

agent_detail = Agent("agent_detaile", description="An agent that provides information **ONLY** regarding user information. Exclusively returns the following information: (cc, food preference, name, address). Do not use for anything else", starting_task="What tools do you have access to")
tools_detail  = [ give_user_info,  send_factory(agent_detail)]
tool_node_detail  = ToolNode(tools_detail )
model_detail = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.2, google_api_key=api_key).bind_tools(tools_detail )
agent_detail.bind_model(model_detail)
workflow_detail = StateGraph(MessagesState)
workflow_detail.add_node("agent", call_factory(agent_detail))
workflow_detail.add_node("tools", tool_node_detail)
workflow_detail.set_entry_point("agent")
workflow_detail.add_edge("agent", "tools")
cycle_stopper = CycleStopper(agent_name="agent", max_cycles=4)
workflow_detail.add_conditional_edges(
    "tools",
    cycle_stopper,
)
checkpointer = MemorySaver()
app = workflow_detail.compile(checkpointer=checkpointer)
agent_detail.bind_workflow(app)


In [ ]:
app.invoke({"messages": "What tools do you have access to"}, config={"configurable": {"thread_id": 2}}  )



 agent_detaile
['What tools do you have access to\n'
 '\n'
 'Current memory:\n'
 'HumanMessage: Recipient_id: agent_detaile | Sender_id: SYSTEM -> What tools '
 'do you have access to']
23





 agent_detaile
['What tools do you have access to\n'
 '\n'
 'Current memory:\n'
 'HumanMessage: Recipient_id: agent_detaile | Sender_id: SYSTEM -> What tools '
 'do you have access to\n'
 'AIMessage: I have access to the following tools:\n'
 '\n'
 '* `give_user_info`: This tool can be used to get information about a '
 'particular user.\n'
 '* `send_message`: This tool can be used to send a message to an agent. \n']
59





 agent_detaile
['What tools do you have access to\n'
 '\n'
 'Current memory:\n'
 'HumanMessage: Recipient_id: agent_detaile | Sender_id: SYSTEM -> What tools '
 'do you have access to\n'
 'AIMessage: I have access to the following tools:\n'
 '\n'
 '* `give_user_info`: This tool can be used to get information about a '
 'particular user.\n'
 '* `send_message`: This tool can 

KeyboardInterrupt: 

In [ ]:
agent_detail.memory_graph.get_memory_messages(agent_detail.agent_id)

[HumanMessage(content='Recipient_id: agent_detaile | Sender_id: SYSTEM -> What tools do you have access to'),
 AIMessage(content='Error: search_agents is not a valid tool, try one of [give_user_info].')]

In [ ]:
agent_sell.memory_graph.get_memory_messages(agent_detail.agent_id)

[]